## Connecting to database and retrieving orderbooks

In [92]:
import psycopg2

fn = '/home/mickael/1-btc_order_book_monitor/db_pwd.secret'
pwd = open(fn, 'r').readlines()[0]
pwd = pwd.strip()

try:
    conn = psycopg2.connect("dbname='cryppro_v0' user='mickael' host='localhost' password='%s'"%pwd)
except:
    print("Unable to connect to the database")
    
cur = conn.cursor()
try:
    cur.execute("""SELECT * from btclobs""")
except:
    print("Unable to SELECT from btclobs table")

rows = cur.fetchall()

## Retrieve each recorded hour's best bid/ask for 10 BTCs limit

In [93]:
import pandas as pd

records=0
rec_var=1
df=pd.DataFrame() #will contain all records
df2=pd.DataFrame() #best hourly bid/ask with 10 BTCs limit
print("number of hours recorded: ", len(rows)/60)

#copying recorded data to df1 dataframe
hrs=int(len(rows)/60)
for i in range(0,hrs):
    df1=pd.DataFrame()
    for l in range (rec_var,rec_var+59):
        if rows[l-1][5]==rows[l][5]:
            df1=df1.append(pd.DataFrame(list(rows[l-1]), columns=[l-1], index=['price','amount','price_pre','side','ticker','timestamp','id']).transpose())
            records=records+1
        else:
            continue
    
    df=df.append(df1)
    if df1.empty:
        continue
    else:          #Sorting data to include best bid/ask by taking orderbook's mid-point
        mid=df1.iloc[int(len(df1)/2)].values
        sum=mid[1]
        df2=df2.append(df1.iloc[int(len(df1)/2)])
        while sum<10: #BTC Limit condition
            df1=df1.drop(df1.index[int(len(df1)/2)])
            mid=df1.iloc[int(len(df1)/2)].values
            sum=sum+mid[1]
            df2=df2.append(df1.iloc[int(len(df1)/2)])
        
    rec_var=rec_var+59    
print("number of records: ", records)

number of hours recorded:  1337.0
number of records:  77569


# Market Maker Simulation

In [ ]:
rec_var=59 #analysis starting point (df row number)
df9=pd.DataFrame() #MM inventory
df9=df9.append(df2.iloc[0:3]) #MM starting inventory
hrs=int(len(rows)/60) #hours recorded

for i in range(0,hrs):
    df1=pd.DataFrame() #hourly order book
    df3=pd.DataFrame() #df1 sorted by best ask/bid    
    
    for l in range (rec_var,rec_var+58):
        df1=df1.append(pd.DataFrame(list(rows[l]), columns=[l], index=['price','amount','price_pre','side','ticker','timestamp','id']).transpose())
    rec_var=rec_var+59    
    while len(df3)<len(df1):    #creates df3 which is similar to df1 but sorted by best bid/ask prices
        maxm=df1[df1['side']=='bid']['price'].max()
        minm=df1[df1['side']=='ask']['price'].min()
        dt1=df1['price']==maxm
        indx1=dt1.index[dt1==True][0]
        dt2=df1['price']==minm
        indx2=dt2.index[dt2==True][0]
        df3=df3.append(df1.loc[df1.index == indx1])
        df1=df1.drop([indx1])
        df3=df3.append(df1.loc[df1.index == indx2])
        df1=df1.drop([indx2])
        
    if df9['amount'][df9['side']=='bid'].sum()>10:   #accumulate only from ask side, if MM bid price is equal or lower than ask price
        for k in range(0,len(df3)):
            if not df3.empty:
                if df3['side'].iloc[0]=='ask':
                    for j in range(0,len(df9)):
                        if df9['side'].iloc[j]=='bid':
                            if df3.iloc[0,0]>=df9['price'].iloc[j]:
                                df9.iloc[j,0]=df9.iloc[j,0]-df3.iloc[0,1]
                                if df9['amount'].iloc[j]>0:
                                    df9=df9.append(df3.iloc[0])
                                    df9.iloc[-1,0]=0
                                if df9['amount'].iloc[j]<0:
                                    df3.iloc[0,1]=df3.iloc[0,1]-df9.iloc[j,0]
                                    df9.iloc[j,0]=0
                                    df9=df9.append(df3.iloc[0])
                                    df9.iloc[-1,0]=0
                df3=df3.iloc[1:]
                             
    if df9['amount'][df9['side']=='ask'].sum()>10:   #accumulate only from bid side, if MM ask price is equal or higher than ask price
        for k in range(0,len(df3)):
            if not df3.empty:
                if df3['side'].iloc[0]=='bid':
                    for j in range(0,len(df9)):
                        if df9['side'].iloc[j]=='ask':
                            if df3.iloc[0,0]<=df9['price'].iloc[j]:
                                if df9['amount'].iloc[j]>0:
                                    df9.iloc[j,0]=df9.iloc[j,0]-df3.iloc[0,1]
                                    if df9['amount'].iloc[j]>0:
                                        df9=df9.append(df3.iloc[0])
                                        df9.iloc[-1,0]=0
                                    if df9['amount'].iloc[j]<0:
                                        df3.iloc[0,1]=df3.iloc[0,1]-df9.iloc[j,0]
                                        df9.iloc[j,0]=0
                                        df9=df9.append(df3.iloc[0])
                                        df9.iloc[-1,0]=0
                df3=df3.iloc[1:]

    else:
        for k in range(0,len(df3)):   # accumulates bid/ask from df3 (already sorted by best bid/ask)
            if df9['amount'][df9['side']=='bid'].sum()<10:
                for j in range(0,len(df9)):
                    if df9['amount'][df9['side']=='bid'].sum()<10:
                        if df3['side'].iloc[0]=='bid':
                            df9=df9.append(df3.iloc[0])
                            df3=df3.iloc[1:]
            if df9['amount'][df9['side']=='ask'].sum()<10:
                for j in range(0,len(df9)):
                    if df9['amount'][df9['side']=='ask'].sum()<10:
                        if df3['side'].iloc[0]=='ask':
                            df9=df9.append(df3.iloc[0])
                            df3=df3.iloc[1:]
    print('Inventory at hour: %.i'%i)
    print('total bid inventory: %.8f'%df9['amount'][df9['side']=='bid'].sum())
    print('total ask inventory: %.8f'%df9['amount'][df9['side']=='ask'].sum())
print('current inventory: ')
print(df9)

Inventory at hour: 0
total bid inventory: 3.44900000
total ask inventory: 10.25400000
Inventory at hour: 1
total bid inventory: 3.44900000
total ask inventory: 10.25400000
Inventory at hour: 2
total bid inventory: 3.44900000
total ask inventory: 0.00000000
Inventory at hour: 3
total bid inventory: 5.20500000
total ask inventory: 10.45400000
Inventory at hour: 4
total bid inventory: 5.20500000
total ask inventory: 0.00000000
Inventory at hour: 5
total bid inventory: 15.10700000
total ask inventory: 6.88100000
Inventory at hour: 6
total bid inventory: 4.52700000
total ask inventory: 6.88100000
Inventory at hour: 7
total bid inventory: 25.89300000
total ask inventory: 10.53400000
Inventory at hour: 8
total bid inventory: 4.52700000
total ask inventory: 10.53400000
Inventory at hour: 9
total bid inventory: 4.52700000
total ask inventory: 3.65300000
Inventory at hour: 10
total bid inventory: 10.48400000
total ask inventory: 5.73100000
Inventory at hour: 11
total bid inventory: 4.52700000
to